In [1]:
import numpy as np

In [2]:
def get_model(save_name='ResNet50.hdh5', image_shape=(224, 224, 3)):
    import os
    if not os.path.exists(save_name):
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(
            include_top=False, weights='imagenet',
            input_tensor=None, input_shape=image_shape, pooling='max'
        )
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model

In [3]:
initial_model = get_model()

Using TensorFlow backend.
/usr/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def get_data(image_shape=(224, 224, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

/usr/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=True
)

In [9]:
last = initial_model.output

nn = Dense(1024, activation='relu')(last)
nn = Dropout(0.5)(nn)
nn = Dense(1024, activation='relu')(nn)

prediction = Dense(num_classes, activation='softmax')(nn)
model = Model(initial_model.input, prediction)

for layer in initial_model.layers:
    layer.trainable = False

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['categorical_accuracy']
)

In [11]:
batch_size = 16
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=300,
    validation_data=(test_x, test_y)
)

Epoch 1/300
109/109 [==============================] - 35s - loss: 3.8309 - categorical_accuracy: 0.0619 - val_loss: 3.9604 - val_categorical_accuracy: 0.0213
Epoch 2/300
109/109 [==============================] - 28s - loss: 3.2072 - categorical_accuracy: 0.1690 - val_loss: 3.9632 - val_categorical_accuracy: 0.0280
Epoch 3/300
109/109 [==============================] - 28s - loss: 2.6799 - categorical_accuracy: 0.2718 - val_loss: 3.9536 - val_categorical_accuracy: 0.0240
Epoch 4/300
109/109 [==============================] - 28s - loss: 2.2567 - categorical_accuracy: 0.3876 - val_loss: 3.9218 - val_categorical_accuracy: 0.0520
Epoch 5/300
109/109 [==============================] - 28s - loss: 1.9483 - categorical_accuracy: 0.4578 - val_loss: 3.8352 - val_categorical_accuracy: 0.0933
Epoch 6/300
109/109 [==============================] - 28s - loss: 1.7660 - categorical_accuracy: 0.4834 - val_loss: 3.4923 - val_categorical_accuracy: 0.1600
Epoch 7/300
109/109 [=========================

109/109 [==============================] - 28s - loss: 0.3499 - categorical_accuracy: 0.8869 - val_loss: 1.4285 - val_categorical_accuracy: 0.6013
Epoch 53/300
109/109 [==============================] - 28s - loss: 0.2760 - categorical_accuracy: 0.9111 - val_loss: 1.4796 - val_categorical_accuracy: 0.6080
Epoch 54/300
109/109 [==============================] - 28s - loss: 0.2808 - categorical_accuracy: 0.9088 - val_loss: 1.4616 - val_categorical_accuracy: 0.6187
Epoch 55/300
109/109 [==============================] - 28s - loss: 0.2496 - categorical_accuracy: 0.9209 - val_loss: 1.4763 - val_categorical_accuracy: 0.6187
Epoch 56/300
109/109 [==============================] - 28s - loss: 0.2805 - categorical_accuracy: 0.9148 - val_loss: 1.4453 - val_categorical_accuracy: 0.6307
Epoch 57/300
109/109 [==============================] - 28s - loss: 0.2446 - categorical_accuracy: 0.9270 - val_loss: 1.5220 - val_categorical_accuracy: 0.6147
Epoch 58/300
109/109 [==============================]

109/109 [==============================] - 28s - loss: 0.1458 - categorical_accuracy: 0.9522 - val_loss: 1.8649 - val_categorical_accuracy: 0.6053
Epoch 104/300
109/109 [==============================] - 28s - loss: 0.1202 - categorical_accuracy: 0.9662 - val_loss: 1.7564 - val_categorical_accuracy: 0.6173
Epoch 105/300
108/109 [============================>.] - ETA: 0s - loss: 0.1549 - categorical_accuracy: 0.9454

KeyboardInterrupt: 

In [ ]:
model.save('0.6_score_model.hdh5')